In [15]:
from datetime import date
from marshmallow import Schema, fields, post_load
from collections import namedtuple

In [5]:
class Person:
    def __init__(self, first_name, last_name, dob, height):
        self.first_name = first_name
        self.last_name = last_name
        self.dob = dob
        self.height = height
        
    def __repr__(self):
        return f'Person({self.first_name}, {self.last_name}, {self.dob})'
    
class PersonSchema(Schema):
    first_name = fields.Str()
    last_name = fields.Str()
    dob = fields.Date()
    height = fields.Int()

p1 = Person('John', 'Cleese', date(1939, 10, 27), 180)
person_schema = PersonSchema()
person_schema.dumps(p1)

'{"first_name": "John", "last_name": "Cleese", "dob": "1939-10-27", "height": 180}'

In [6]:
PT=namedtuple('PT', 'first_name, last_name, dob, height')
p2 = PT('Eric', 'Idle', date(1943, 3, 29), 180)
person_schema.dumps(p2)

'{"first_name": "Eric", "last_name": "Idle", "dob": "1943-03-29", "height": 180}'

In [7]:
PT2 = namedtuple('PT2', 'first_name, last_name, age')
p3 = PT2('Michael', 'Palin', 75)
person_partial = PersonSchema(only=('first_name', 'last_name'))
person_partial.dumps(p3)

'{"first_name": "Michael", "last_name": "Palin"}'

In [11]:
PT2 = namedtuple('PT2', 'first_name, last_name, height')
p3 = PT2('Michael', 'Palin', 75)
person_partial = PersonSchema(exclude=['height'])
person_partial.dumps(p3)

'{"first_name": "Michael", "last_name": "Palin"}'

In [13]:
class Person:
    def __init__(self, first_name, last_name, dob):
        self.first_name = first_name
        self.last_name = last_name
        self.dob = dob
        
    def __repr__(self):
        return f'Person({self.first_name}, {self.last_name}, {self.dob})'
    
class PersonSchema(Schema):
    first_name = fields.Str()
    last_name = fields.Str()
    dob = fields.Date()

class Movie:
    def __init__(self, title, year, actors):
        self.title = title
        self.year = year
        self.actors = actors
        
class MovieSchema(Schema):
    title = fields.Str()
    year = fields.Integer()
    actors = fields.Nested(PersonSchema, many=True)
    
p1 = Person('John', 'Cleese', date(1939, 10, 27))
p2 = Person('Michael', 'Palin', date(1943, 5, 5))


parrot = Movie('Parrot Sketch', 1989, [p1, p2])

MovieSchema().dumps(parrot)

'{"year": 1989, "title": "Parrot Sketch", "actors": [{"first_name": "John", "last_name": "Cleese", "dob": "1939-10-27"}, {"first_name": "Michael", "last_name": "Palin", "dob": "1943-05-05"}]}'

# Deserialization

In [26]:
class Person:
    def __init__(self, first_name, last_name, dob):
        self.first_name = first_name
        self.last_name = last_name
        self.dob = dob
      
    def __repr__(self):
        return f'Person({self.first_name}, {self.last_name}, {self.dob})'

class PersonSchema(Schema):
    first_name = fields.Str()
    last_name = fields.Str()
    dob = fields.Date()
    
    @post_load
    def make_person(self, data, **kwargs):
        return Person(**data)
    
person_schema = PersonSchema()
p_dict = dict(first_name='John', last_name='Cleese', dob='1939-10-27')
person_schema.load(p_dict)

Person(John, Cleese, 1939-10-27)

In [44]:
class MovieSchema(Schema):
    title = fields.Str()
    year = fields.Integer()
    actors = fields.Nested(PersonSchema, many=True)
    
    @post_load
    def make_movie(self, data, **kwargs):
        return Movie(**data)
    
json_data = '''
{"actors": [
    {"first_name": "John", "last_name": "Cleese", "dob": "1939-10-27"}, 
    {"first_name": "Michael", "last_name": "Palin", "dob": "1943-05-05"}], 
"title": "Parrot Sketch", 
"year": 1989}
'''
movie_schema = MovieSchema()
movie = movie_schema.loads(json_data)

print(movie.title)
print(movie.actors)
print(movie.year)

Parrot Sketch
[Person(John, Cleese, 1939-10-27), Person(Michael, Palin, 1943-05-05)]
1989
